<a href="https://colab.research.google.com/github/MarinaChau/DataViz-Project-2022/blob/main/DataViz_Food.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project Data viz

In [61]:


# Import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# plotly
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import iplot, plot
from plotly.subplots import make_subplots
## fixer le theme
import plotly.io as pio
pio.templates.default = 'ggplot2'

# scikit-learn
## Métriques
from sklearn.metrics import silhouette_score
## Clustering
from sklearn.cluster import KMeans

# yellowbrick
## Clustering
from yellowbrick.cluster import SilhouetteVisualizer

In [2]:
# Download the different dataset

!wget https://www.data.gouv.fr/fr/datasets/r/83289c89-8134-4426-8959-eccad79f8873 # Summary of the dataset

--2022-06-24 06:49:48--  https://www.data.gouv.fr/fr/datasets/r/83289c89-8134-4426-8959-eccad79f8873
Resolving www.data.gouv.fr (www.data.gouv.fr)... 37.59.183.93
Connecting to www.data.gouv.fr (www.data.gouv.fr)|37.59.183.93|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://koumoul.com/data-fair/api/v1/datasets/agribalyse-synthese/raw [following]
--2022-06-24 06:49:49--  https://koumoul.com/data-fair/api/v1/datasets/agribalyse-synthese/raw
Resolving koumoul.com (koumoul.com)... 51.75.192.4
Connecting to koumoul.com (koumoul.com)|51.75.192.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 927255 (906K) [text/csv]
Saving to: ‘83289c89-8134-4426-8959-eccad79f8873’

83289c89-8134-4426- 100%[===================>] 905.52K  1.52MB/s    in 0.6s    

2022-06-24 06:49:49 (1.52 MB/s) - ‘83289c89-8134-4426-8959-eccad79f8873’ saved [927255/927255]



In [84]:
df = pd.read_csv("/content/synthese.csv")

In [85]:
# We're going to drop "Code AGB" and "Code CIQUAL"
df = df.drop(['Code AGB', 'Code CIQUAL', 'LCI Name'], axis=1)

As the global population is eating more and more meat, the CO2 impact is also growing. In a time where it is important to understand and reduce our CO2 impact individually, it is essential to inform people on their diet and its environmental impact.



In [64]:
df.columns

Index(['Groupe d'aliment', 'Sous-groupe d'aliment',
       'Nom du Produit en Français', 'Saisonnalité',
       'Transport par avion (1 : par avion)', 'Livraison',
       'Matériau d'emballage', 'Préparation',
       'DQR - Note de qualité de la donnée (1 excellente ; 5 très faible)',
       'Score unique EF (mPt/kg de produit)',
       'Changement climatique (kg CO2 eq/kg de produit)',
       'Appauvrissement de la couche d'ozone (E-06 kg CVC11 eq/kg de produit)',
       'Rayonnements ionisants (kBq U-235 eq/kg de produit)',
       'Formation photochimique d'ozone (E-03 kg NMVOC eq/kg de produit)',
       'Particules (E-06 disease inc./kg de produit)',
       'Acidification terrestre et eaux douces (mol H+ eq/kg de produit)',
       'Eutrophisation terreste (mol N eq/kg de produit)',
       'Eutrophisation eaux douces (E-03 kg P eq/kg de produit)',
       'Eutrophisation marine (E-03 kg N eq/kg de produit)',
       'Utilisation du sol (Pt/kg de produit)',
       'Écotoxicité pour écos

## 1. Environmental impact in CO2 eq per type of product

In [65]:
target = "Changement climatique (kg CO2 eq/kg de produit)"

In [66]:
fig = px.treemap(df, path=["Groupe d'aliment"], 
                values=target,
                color=target,
                color_continuous_scale=px.colors.diverging.Portland,
                color_continuous_midpoint=3,
                labels={"Changement climatique (kg CO2 eq/kg de produit)":"Kg CO2 eq"},
                title="Changement climatique (kg CO2 eq/kg de produit)"
                )
fig


## 2. Water quantity used for producing the "food"

In [67]:
df_food_cat = df.groupby(by="Groupe d'aliment").agg(["mean"])
x = df_food_cat["Épuisement des ressources eau (m3 depriv./kg de produit)"].values.tolist()
el_values = [el[0] for el in x]

In [68]:

labels = df_food_cat.index.values.tolist()
values = el_values

# pull is given as a fraction of the pie radius

fig = go.Figure(data=[go.Pie(labels=labels, values=values, pull=[0, 0, 0, 0, 0, 0, 0, 0,0,0.2, 0])])
fig.update_layout(
    title_text="Épuisement des ressources eau (m3 depriv./kg de produit)")
fig.show()

## Water pollution


In [69]:
df_group = df.groupby("Groupe d'aliment").agg("mean")
df_group.head()

,Transport par avion (1 : par avion),DQR - Note de qualité de la donnée (1 excellente ; 5 très faible),Score unique EF (mPt/kg de produit),Changement climatique (kg CO2 eq/kg de produit),Appauvrissement de la couche d'ozone (E-06 kg CVC11 eq/kg de produit),Rayonnements ionisants (kBq U-235 eq/kg de produit),Formation photochimique d'ozone (E-03 kg NMVOC eq/kg de produit),Particules (E-06 disease inc./kg de produit),Acidification terrestre et eaux douces (mol H+ eq/kg de produit),Eutrophisation terreste (mol N eq/kg de produit),Eutrophisation eaux douces (E-03 kg P eq/kg de produit),Eutrophisation marine (E-03 kg N eq/kg de produit),Utilisation du sol (Pt/kg de produit),Écotoxicité pour écosystèmes aquatiques d'eau douce (CTUe/kg de produit),Épuisement des ressources eau (m3 depriv./kg de produit),Épuisement des ressources énergétiques (MJ/kg de produit),Épuisement des ressources minéraux (E-06 kg Sb eq/kg de produit)
Groupe d'aliment,,,,,,,,,,,,,,,,,
aides culinaires et ingrédients divers,0.000000,3.032468,0.389715,2.618907,0.407491,2.031872,9.069066,0.210587,0.027481,0.077491,0.693687,14.410384,130.977090,81.571944,3.510876,65.041364,7.630547
aliments infantiles,0.000000,2.804545,0.304532,2.561923,0.298016,0.808958,8.037662,0.189487,0.026472,0.102830,0.351380,18.242204,148.917146,53.975172,2.309336,32.005056,3.085692
boissons,0.000000,2.981233,0.187846,1.501865,0.151390,0.275726,4.609922,0.104814,0.012148,0.043902,0.312013,6.471281,43.014891,78.833041,2.052653,16.607615,3.637176
entrées et plats composés,0.000000,2.451779,0.601687,4.913607,0.422651,1.420043,20.261443,0.511242,0.073075,0.284347,0.758012,20.906551,252.313648,82.284161,3.212115,53.665766,6.285330
"fruits, légumes, légumineuses et oléagineux",0.004728,2.891040,0.287862,1.556569,0.878225,1.023588,5.547926,0.112894,0.014317,0.053970,0.443338,8.277514,85.901820,81.900307,7.990790,34.838141,5.633235


In [70]:
df_group.index

Index(['aides culinaires et ingrédients divers', 'aliments infantiles',
       'boissons', 'entrées et plats composés',
       'fruits, légumes, légumineuses et oléagineux', 'glaces et sorbets',
       'lait et produits laitiers', 'matières grasses', 'produits céréaliers',
       'produits sucrés', 'viandes, œufs, poissons'],
      dtype='object', name='Groupe d'aliment')

In [71]:

fig = px.bar(df_group, x=df_group.index, y="Eutrophisation eaux douces (E-03 kg P eq/kg de produit)")
fig.show()

## It's not only the production, it is also how it is packaged!

In [72]:
df["Matériau d'emballage"].value_counts()

PS                          516
LPDE                        479
PP                          387
Carton                      304
Pas d'emballage             156
Déjà emballé - PET          155
Acier                       107
Verre                        92
V (PVC)                      81
Bouteille PETE               67
Papier                       37
HPDE                         31
Déjà emballé - PP/PE         27
Déjà emballé - Aluminium     16
Déjà emballé - Verre         14
Déjà emballé - LDPE           7
Aluminium                     1
PVC                           1
Bouteille PET                 1
Name: Matériau d'emballage, dtype: int64

In [73]:
plastique_emballage = ['PS', 'LPDE', 'PP', 'Déjà emballé - PET', 'V (PVC)', 'Bouteille PETE',  'HPDE',
                       'Déjà emballé - PP/PE', 'Déjà emballé - LDPE', 'PVC', 'Bouteille PET',]

# on conserve que les produits contenant ces emballages
df_plastique = df[(df.loc[:,["Matériau d'emballage"]].isin(plastique_emballage)).squeeze()]


table = df.loc[:,["Matériau d'emballage"]].isin(plastique_emballage)
len(table[table["Matériau d'emballage"]==True])/len(df)

0.7067365873336022

In [74]:

TARGET = "Changement climatique (kg CO2 eq/kg de produit)"

In [86]:
 df["Matériau d'emballage"][df["Matériau d'emballage"].isin(plastique_emballage)] = "PLASTIQUE"

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [87]:
df

,Groupe d'aliment,Sous-groupe d'aliment,Nom du Produit en Français,Saisonnalité,Transport par avion (1 : par avion),Livraison,Matériau d'emballage,Préparation,DQR - Note de qualité de la donnée (1 excellente ; 5 très faible),Score unique EF (mPt/kg de produit),...,Particules (E-06 disease inc./kg de produit),Acidification terrestre et eaux douces (mol H+ eq/kg de produit),Eutrophisation terreste (mol N eq/kg de produit),Eutrophisation eaux douces (E-03 kg P eq/kg de produit),Eutrophisation marine (E-03 kg N eq/kg de produit),Utilisation du sol (Pt/kg de produit),Écotoxicité pour écosystèmes aquatiques d'eau douce (CTUe/kg de produit),Épuisement des ressources eau (m3 depriv./kg de produit),Épuisement des ressources énergétiques (MJ/kg de produit),Épuisement des ressources minéraux (E-06 kg Sb eq/kg de produit)
0,"fruits, légumes, légumineuses et oléagineux",fruits,"Abricot au sirop léger, appertisé, égoutté",mix de consommation FR,0,Ambiant (moyenne),Acier,Pas de préparation,2.46,0.215366,...,0.096413,0.009301,0.031766,0.410555,1.502775,44.823181,62.195218,1.630637,47.237697,2.938064
1,"fruits, légumes, légumineuses et oléagineux",fruits,"Abricot au sirop léger, appertisé, non égoutté",mix de consommation FR,0,Ambiant (moyenne),Acier,Pas de préparation,2.46,0.128043,...,0.059314,0.005726,0.019499,0.235366,0.797791,27.901738,36.285914,1.010960,29.397697,1.823726
2,"fruits, légumes, légumineuses et oléagineux",fruits,"Abricot au sirop, appertisé, égoutté",mix de consommation FR,0,Ambiant (moyenne),Acier,Pas de préparation,2.46,0.215366,...,0.096413,0.009301,0.031766,0.410555,1.502775,44.823181,62.195218,1.630637,47.237697,2.938064
3,"fruits, légumes, légumineuses et oléagineux",fruits,"Abricot au sirop, appertisé, non égoutté",mix de consommation FR,0,Ambiant (moyenne),Acier,Pas de préparation,2.44,0.128043,...,0.059314,0.005726,0.019499,0.235366,0.797791,27.901738,36.285914,1.010960,29.397697,1.823726
4,"fruits, légumes, légumineuses et oléagineux",fruits,"Abricot, dénoyauté, cru",mix de consommation FR,0,Ambiant (moyenne),PLASTIQUE,Pas de préparation,2.20,0.118929,...,0.061462,0.007254,0.025954,0.204244,0.943669,43.838644,41.957928,1.412878,17.123820,2.173328
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2474,lait et produits laitiers,produits laitiers frais et assimilés,"Yaourt, lait fermenté ou spécialité laitière, ...",mix de consommation FR,0,Glacé,PLASTIQUE,Pas de préparation,3.61,0.351296,...,0.173049,0.022129,0.092536,0.257436,15.198522,136.090810,216.954380,2.253106,26.865322,5.020883
2475,lait et produits laitiers,produits laitiers frais et assimilés,"Yaourt, lait fermenté ou spécialité laitière, ...",mix de consommation FR,0,Glacé,PLASTIQUE,Pas de préparation,2.04,0.239814,...,0.155803,0.022678,0.095881,0.235488,13.642379,133.154710,24.979408,0.636518,25.746521,1.838004
2476,lait et produits laitiers,produits laitiers frais et assimilés,"Yaourt, lait fermenté ou spécialité laitière, ...",mix de consommation FR,0,Glacé,PLASTIQUE,Pas de préparation,2.48,0.239814,...,0.155803,0.022678,0.095881,0.235488,13.642379,133.154710,24.979408,0.636518,25.746521,1.838004
2477,lait et produits laitiers,produits laitiers frais et assimilés,"Yaourt, lait fermenté ou spécialité laitière, ...",mix de consommation FR,0,Glacé,PLASTIQUE,Pas de préparation,1.90,0.260504,...,0.165113,0.023417,0.097687,0.297081,14.777646,138.486420,32.692803,0.862339,27.136623,2.858934


In [88]:
# liste des variables type 'object'  
col = "Matériau d'emballage"

# afficher les box-plot

fig = px.box(df, x=col, y=target, color=col, 
             title="CO2 eq/kg de produit en fonction de '{}'".format(col),
             color_discrete_sequence=px.colors.qualitative.Vivid
            )
fig.add_hline(y=3, line_width=1, line_color="red")
fig.update_layout(boxgap=0, showlegend=False)
fig.update_xaxes(tickangle=45)
fig.update_traces(boxmean=True)
iplot(fig)

what kind of food are mostly using plastic? 

In [90]:
df_grp = df.groupby(by="Groupe d'aliment").agg("mean")

In [93]:
df_grp.index

Index(['aides culinaires et ingrédients divers', 'aliments infantiles',
       'boissons', 'entrées et plats composés',
       'fruits, légumes, légumineuses et oléagineux', 'glaces et sorbets',
       'lait et produits laitiers', 'matières grasses', 'produits céréaliers',
       'produits sucrés', 'viandes, œufs, poissons'],
      dtype='object', name='Groupe d'aliment')

## It is also how the product is deliver to you!

In [98]:
fig = px.violin(df, x="Changement climatique (kg CO2 eq/kg de produit)",
                        y="Livraison", orientation='h', color="Livraison",
                        #title="{} en fonction de la variable {}".format(col1,col2)
                        color_discrete_sequence=px.colors.qualitative.Vivid
                       )
fig.update_traces(side='positive', width=2)
fig.update_layout(showlegend=False)
fig.update_layout(showlegend=False)
iplot(fig)

In [ ]:
## What is the category of food with the most representation in plastic and "congelé" product?

In [ ]:
df = px.data.tips()
fig = px.histogram(df, x="sex", y="total_bill",
             color='smoker', barmode='group',
             histfunc='avg',
             height=400)
fig.show()

## Now! Talking about the menu!

In [99]:
df.columns


Index(['Groupe d'aliment', 'Sous-groupe d'aliment',
       'Nom du Produit en Français', 'Saisonnalité',
       'Transport par avion (1 : par avion)', 'Livraison',
       'Matériau d'emballage', 'Préparation',
       'DQR - Note de qualité de la donnée (1 excellente ; 5 très faible)',
       'Score unique EF (mPt/kg de produit)',
       'Changement climatique (kg CO2 eq/kg de produit)',
       'Appauvrissement de la couche d'ozone (E-06 kg CVC11 eq/kg de produit)',
       'Rayonnements ionisants (kBq U-235 eq/kg de produit)',
       'Formation photochimique d'ozone (E-03 kg NMVOC eq/kg de produit)',
       'Particules (E-06 disease inc./kg de produit)',
       'Acidification terrestre et eaux douces (mol H+ eq/kg de produit)',
       'Eutrophisation terreste (mol N eq/kg de produit)',
       'Eutrophisation eaux douces (E-03 kg P eq/kg de produit)',
       'Eutrophisation marine (E-03 kg N eq/kg de produit)',
       'Utilisation du sol (Pt/kg de produit)',
       'Écotoxicité pour écos

In [100]:
df["Sous-groupe d'aliment"].value_counts()

légumes                                              196
charcuteries                                         145
viandes crues                                        137
céréales de petit-déjeuner et biscuits               125
plats composés                                       122
fromages                                             117
viandes cuites                                       105
fruits                                                96
poissons crus                                         94
eaux                                                  90
boissons sans alcool                                  83
produits laitiers frais et assimilés                  80
gâteaux et pâtisseries                                76
pains et viennoiseries                                73
pâtes, riz et céréales                                60
sauces                                                58
poissons cuits                                        55
fruits à coque et graines oléag

In [102]:
df_ss_grp = df.groupby("Sous-groupe d'aliment").agg("mean")
df_ss_grp.head()

,Transport par avion (1 : par avion),DQR - Note de qualité de la donnée (1 excellente ; 5 très faible),Score unique EF (mPt/kg de produit),Changement climatique (kg CO2 eq/kg de produit),Appauvrissement de la couche d'ozone (E-06 kg CVC11 eq/kg de produit),Rayonnements ionisants (kBq U-235 eq/kg de produit),Formation photochimique d'ozone (E-03 kg NMVOC eq/kg de produit),Particules (E-06 disease inc./kg de produit),Acidification terrestre et eaux douces (mol H+ eq/kg de produit),Eutrophisation terreste (mol N eq/kg de produit),Eutrophisation eaux douces (E-03 kg P eq/kg de produit),Eutrophisation marine (E-03 kg N eq/kg de produit),Utilisation du sol (Pt/kg de produit),Écotoxicité pour écosystèmes aquatiques d'eau douce (CTUe/kg de produit),Épuisement des ressources eau (m3 depriv./kg de produit),Épuisement des ressources énergétiques (MJ/kg de produit),Épuisement des ressources minéraux (E-06 kg Sb eq/kg de produit)
Sous-groupe d'aliment,,,,,,,,,,,,,,,,,
aides culinaires,0.0,2.943333,0.850646,3.464760,1.204335,11.663065,10.306611,0.231765,0.025386,0.082147,1.026901,14.040957,72.774083,145.228580,6.318813,266.910904,10.697491
algues,0.0,2.965625,1.253544,6.550417,1.894681,12.273790,34.826934,0.701361,0.098453,0.116204,2.242201,11.708530,21.760145,158.083206,4.934366,341.807783,30.245576
autres matières grasses,0.0,2.755000,0.201385,1.449861,0.155604,0.507139,4.021885,0.169020,0.023686,0.098339,0.283277,5.517637,66.727710,33.392402,1.212132,22.812050,2.323882
autres produits à base de viande,0.0,2.575714,1.762381,16.886849,0.684039,2.334004,28.620531,1.636405,0.240231,1.045362,1.540318,59.350184,1113.597387,217.671252,6.968183,93.677413,11.132867
beurres,0.0,3.314000,1.151625,15.491982,0.514093,0.797715,25.423341,0.841915,0.110323,0.456672,1.920928,37.615043,481.879142,97.260257,9.942513,56.243421,17.771127


In [ ]:

df_bg = pd.concat([df_ss_grp.loc["fruits à coque et graines oléagineuses"], df_ss_grp.loc["légumineuses"], 
                     df_ss_grp.loc["légumes"], df_ss_grp.loc["pâtes, riz et céréales"], 
                     df_ss_grp.loc["produits laitiers frais et assimilés"],
                     df_ss_grp.loc["fromages"],
                     df_ss_grp.loc["viandes crues"],
                     df_ss_grp.loc["poissons crus"],
                   df_ss_grp.loc["mollusques et crustacés crus"],
                   df_ss_grp.loc["viandes cuites"],
                   df_ss_grp.loc["poissons cuits"],
                     ], axis=1)

In [152]:
#  fruits à coque et graines oléagineuses  + légumineuses  + légumes + pâtes, riz et céréales   
temp = ["produits laitiers frais et assimilés", "fromages", "viandes crues",
        "poissons crus", "mollusques et crustacés crus", "viandes cuites",
        "poissons cuits"]

df_vegan = pd.concat([df_ss_grp.loc["fruits à coque et graines oléagineuses"], df_ss_grp.loc["légumineuses"], 
                     df_ss_grp.loc["légumes"], df_ss_grp.loc["pâtes, riz et céréales"], 
                     df_ss_grp.loc["produits laitiers frais et assimilés"],
                     df_ss_grp.loc["fromages"],
                     df_ss_grp.loc["viandes crues"],
                     df_ss_grp.loc["poissons crus"],
                   df_ss_grp.loc["mollusques et crustacés crus"],
                   df_ss_grp.loc["viandes cuites"],
                   df_ss_grp.loc["poissons cuits"]], axis=1)
df_vegan.loc["Changement climatique (kg CO2 eq/kg de produit)"]

df_vegan = pd.DataFrame(df_vegan.loc["Changement climatique (kg CO2 eq/kg de produit)"])
df_vegan["Régime"] = "Vegan"
df_vegan.reset_index(drop=False, inplace=True)
df_vegan.columns = df_avg.columns.str.replace('index', 'cat')
df_vegan["Changement climatique (kg CO2 eq/kg de produit)"][df_vegan["cat"].isin(temp)] = 0
df_vegan

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,cat,Changement climatique (kg CO2 eq/kg de produit),Régime
0,fruits à coque et graines oléagineuses,4.151597,Vegan
1,légumineuses,0.614707,Vegan
2,légumes,1.267678,Vegan
3,"pâtes, riz et céréales",1.456732,Vegan
4,produits laitiers frais et assimilés,0.000000,Vegan
5,fromages,0.000000,Vegan
6,viandes crues,0.000000,Vegan
7,poissons crus,0.000000,Vegan
8,mollusques et crustacés crus,0.000000,Vegan
9,viandes cuites,0.000000,Vegan


In [153]:
#  fruits à coque et graines oléagineuses  + légumineuses  + légumes + pâtes, riz et céréales   
temp = ["viandes crues",
        "poissons crus", "mollusques et crustacés crus", "viandes cuites",
        "poissons cuits"]

df_vege = pd.concat([df_ss_grp.loc["fruits à coque et graines oléagineuses"], df_ss_grp.loc["légumineuses"], 
                     df_ss_grp.loc["légumes"], df_ss_grp.loc["pâtes, riz et céréales"], 
                     df_ss_grp.loc["produits laitiers frais et assimilés"],
                     df_ss_grp.loc["fromages"],
                     df_ss_grp.loc["viandes crues"],
                     df_ss_grp.loc["poissons crus"],
                   df_ss_grp.loc["mollusques et crustacés crus"],
                   df_ss_grp.loc["viandes cuites"],
                   df_ss_grp.loc["poissons cuits"]], axis=1)
df_vege.loc["Changement climatique (kg CO2 eq/kg de produit)"]

df_vege = pd.DataFrame(df_vege.loc["Changement climatique (kg CO2 eq/kg de produit)"])
df_vege["Régime"] = "végétarien"
df_vege.reset_index(drop=False, inplace=True)
df_vege.columns = df_avg.columns.str.replace('index', 'cat')
df_vege["Changement climatique (kg CO2 eq/kg de produit)"][df_vege["cat"].isin(temp)] = 0
df_vege

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,cat,Changement climatique (kg CO2 eq/kg de produit),Régime
0,fruits à coque et graines oléagineuses,4.151597,végétarien
1,légumineuses,0.614707,végétarien
2,légumes,1.267678,végétarien
3,"pâtes, riz et céréales",1.456732,végétarien
4,produits laitiers frais et assimilés,2.923881,végétarien
5,fromages,6.063200,végétarien
6,viandes crues,0.000000,végétarien
7,poissons crus,0.000000,végétarien
8,mollusques et crustacés crus,0.000000,végétarien
9,viandes cuites,0.000000,végétarien


In [155]:
temp = [ "mollusques et crustacés crus", "viandes cuites",
        "poissons cuits"]

df_avg = pd.concat([df_ss_grp.loc["fruits à coque et graines oléagineuses"], df_ss_grp.loc["légumineuses"], 
                     df_ss_grp.loc["légumes"], df_ss_grp.loc["pâtes, riz et céréales"], 
                     df_ss_grp.loc["produits laitiers frais et assimilés"],
                     df_ss_grp.loc["fromages"],
                     df_ss_grp.loc["viandes crues"],
                     df_ss_grp.loc["poissons crus"],
                   df_ss_grp.loc["mollusques et crustacés crus"],
                   df_ss_grp.loc["viandes cuites"],
                   df_ss_grp.loc["poissons cuits"],
                     ], axis=1)
df_avg.loc["Changement climatique (kg CO2 eq/kg de produit)"],

df_avg = pd.DataFrame(df_avg.loc["Changement climatique (kg CO2 eq/kg de produit)"])
df_avg["Régime"] = "mangeur AVG"
df_avg.reset_index(drop=False, inplace=True)
df_avg.columns = df_avg.columns.str.replace('index', 'cat')
df_avg["Changement climatique (kg CO2 eq/kg de produit)"][df_avg["cat"].isin(temp)] = 0
df_avg

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,cat,Changement climatique (kg CO2 eq/kg de produit),Régime
0,fruits à coque et graines oléagineuses,4.151597,mangeur AVG
1,légumineuses,0.614707,mangeur AVG
2,légumes,1.267678,mangeur AVG
3,"pâtes, riz et céréales",1.456732,mangeur AVG
4,produits laitiers frais et assimilés,2.923881,mangeur AVG
5,fromages,6.063200,mangeur AVG
6,viandes crues,18.269879,mangeur AVG
7,poissons crus,7.545905,mangeur AVG
8,mollusques et crustacés crus,0.000000,mangeur AVG
9,viandes cuites,0.000000,mangeur AVG


In [139]:
df_bg = pd.concat([df_ss_grp.loc["fruits à coque et graines oléagineuses"], df_ss_grp.loc["légumineuses"], 
                     df_ss_grp.loc["légumes"], df_ss_grp.loc["pâtes, riz et céréales"], 
                     df_ss_grp.loc["produits laitiers frais et assimilés"],
                     df_ss_grp.loc["fromages"],
                     df_ss_grp.loc["viandes crues"],
                     df_ss_grp.loc["poissons crus"],
                   df_ss_grp.loc["mollusques et crustacés crus"],
                   df_ss_grp.loc["viandes cuites"],
                   df_ss_grp.loc["poissons cuits"],
                     ], axis=1)
df_bg.loc["Changement climatique (kg CO2 eq/kg de produit)"],

df_4 = pd.DataFrame(df_bg.loc["Changement climatique (kg CO2 eq/kg de produit)"])
df_4["Régime"] = "Grand mangeur de produits animals"
df_4.reset_index(drop=False, inplace=True)
df_4.columns = df_4.columns.str.replace('index', 'cat')
df_4

,cat,Changement climatique (kg CO2 eq/kg de produit),Régime
0,fruits à coque et graines oléagineuses,4.151597,Grand mangeur de produits animals
1,légumineuses,0.614707,Grand mangeur de produits animals
2,légumes,1.267678,Grand mangeur de produits animals
3,"pâtes, riz et céréales",1.456732,Grand mangeur de produits animals
4,produits laitiers frais et assimilés,2.923881,Grand mangeur de produits animals
5,fromages,6.063200,Grand mangeur de produits animals
6,viandes crues,18.269879,Grand mangeur de produits animals
7,poissons crus,7.545905,Grand mangeur de produits animals
8,mollusques et crustacés crus,9.272484,Grand mangeur de produits animals
9,viandes cuites,28.430414,Grand mangeur de produits animals


In [157]:

df_summary = pd.concat([df_vegan, df_vege, df_4, df_avg])
df_summary

,cat,Changement climatique (kg CO2 eq/kg de produit),Régime
0,fruits à coque et graines oléagineuses,4.151597,Vegan
1,légumineuses,0.614707,Vegan
2,légumes,1.267678,Vegan
3,"pâtes, riz et céréales",1.456732,Vegan
4,produits laitiers frais et assimilés,0.000000,Vegan
5,fromages,0.000000,Vegan
6,viandes crues,0.000000,Vegan
7,poissons crus,0.000000,Vegan
8,mollusques et crustacés crus,0.000000,Vegan
9,viandes cuites,0.000000,Vegan


In [160]:

fig = px.bar(df_summary, x="Régime", y="Changement climatique (kg CO2 eq/kg de produit)", color="cat")
fig.show()